In [1]:
from web3 import Web3
from solcx import install_solc, compile_standard
import solcx
import json

install_solc(version='0.8.0')

onchain = Web3(Web3.HTTPProvider('http://localhost:8545')) #on-chain

s1 = Web3(Web3.HTTPProvider('http://localhost:8546')) #shard 1
s2 = Web3(Web3.HTTPProvider('http://localhost:8547')) #shard 2
s3 = Web3(Web3.HTTPProvider('http://localhost:8548')) #shard 3

onchain.isConnected(), s1.isConnected(), s2.isConnected(), s3.isConnected()

(True, False, False, False)

In [3]:
# Leggo il file contenente lo smart contract
with open("prova_on_chain.sol", "r") as file:
    prova_on_chain_file = file.read()

# Compilo il file contenente lo smart contract
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"prova_on_chain.sol": {"content": prova_on_chain_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"] # output needed to interact with and deploy contract 
                }
            }
        },
    },
    solc_version="0.8.0",
)

# get bytecode
bytecode = compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["evm"]["bytecode"]["object"]
# get abi
abi = json.loads(compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["metadata"])["output"]["abi"]

# Seleziono l'account con cui effettuare le transazioni
onchain.eth.default_account = onchain.eth.accounts[0]

On_Chain_Manager = onchain.eth.contract(abi=abi, bytecode=bytecode)
tx_hash = On_Chain_Manager.constructor().transact()
tx_receipt = onchain.eth.wait_for_transaction_receipt(tx_hash)

on_chain_manager = onchain.eth.contract(
     address=tx_receipt.contractAddress,
     abi=abi
)

print(type(tx_receipt.contractAddress))

on_chain_manager.functions.get_shard_where_deploy().call()


<class 'str'>


1